# Vadere ema connector demo
For more information on the use of the EMA Workbech, please refer to the [official EMA Workbench documentation](https://emaworkbench.readthedocs.io/en/latest/).

## Step 1: imports
The first step is to import the needed modules. This depends on the use and the type of analysis that is intended. The most important one here is the VadereModel from the model connectors. As said, please refer for more information on this to the [official EMA Workbench documentation](https://emaworkbench.readthedocs.io/en/latest/).

In [1]:
from ema_workbench import (perform_experiments, RealParameter, ema_logging,
                           CategoricalParameter, MultiprocessingEvaluator,
                           ScalarOutcome, IntegerParameter)
from ema_workbench.em_framework.parameters import Category
from ema_workbench.connectors.vadere import VadereModel
import pandas as pd

/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/em_framework/evaluators.py:21: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn(
/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/em_framework/optimization.py:48: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)
/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/connectors/__init__.py:17: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/connectors/__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("ne

## Step 2: Setting up the model
In this example we use the Vadere model from this research.  

In [2]:
# This model saves scalar results to a density.txt and speed.txt file.
model = VadereModel('model', 
                    vadere_jar='vadere-console.jar',
                    processor_files=[
                        'density.txt',
                        'speed.txt'
                    ],
                    model_file='base_case.scenario',
                    wd='/media/veracrypt1/thesis/thesis-drive/model/connector/output')

Note that for specifying model uncertainties (and potential levers), the Vadere model class can change any variable present in the model file (Vadere scenario). To realise this, a exact location to the variable of interest in the Vadere scenario file has to be specified. Vadere scenario files follow a nested dictionary structure. Therefore, the exact location of the variable should be passed in a list of argumentes, passed as one string. See the example below, that variates the spawnNumber and maxSpawnNumber of source 0 and source 1 in the Vadere model.

In [3]:
model.uncertainties = [
    IntegerParameter(
        name='pedestrian flow source 0',
        lower_bound=50,
        upper_bound=100,
        variable_name=[
            '("scenario", "topography", "sources", 0, "spawnNumber")',
            '("scenario", "topography", "sources", 0, "maxSpawnNumberTotal")'
        ]
    ),
    IntegerParameter(
        name='pedestrian flow source 1',
        lower_bound=20,
        upper_bound=30,
        variable_name=[
            '("scenario", "topography", "sources", 1, "spawnNumber")',
            '("scenario", "topography", "sources", 1, "maxSpawnNumberTotal")'
        ]
    )
]

The model outcomes can be specified by passing the exact name as present in the output file (speed.txt here). The naming convention depends on the used Vadere data processors, but usally follows the name + id of the processor. When in doubt, it is advised to do a demo Vadere run using the Vadere software and to inspect the generated output files.

In [4]:
model.outcomes = [
    ScalarOutcome('mean_area_speed_processor-PID4'),
]

## Step 3: Performing experiments
The last step is to perform experiment with the Vadere model. Both sequential runs as runs in parallel are supported. Note however that a Vadere run can use a lot of RAM, and using all available cpu cors can lead to performance issues in some cases. 

In [5]:
# enable EMA logging
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [6]:
# run in sequential 2 experiments
results_sequential = perform_experiments(model, 2)

[MainProcess/INFO] performing 2 scenarios * 1 policies * 1 model(s) = 2 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] experiments finished


In [7]:
# run 4 experiments in parallel
with MultiprocessingEvaluator(model, n_processes=4) as evaluator:
        results_parallel = evaluator.perform_experiments(4)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 4 scenarios * 1 policies * 1 model(s) = 4 experiments
[ForkPoolWorker-3/INFO] finalizing
[ForkPoolWorker-4/INFO] finalizing
[ForkPoolWorker-2/INFO] finalizing
Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/geodan-florisb/anaconda3/envs/CrowdSim/lib/python3.10/site-packages/ema_workbench-2.0.10-py3.10.egg/ema_workbench/em_framework/ema_multiprocessing.py", line 168, in worker
    return experiment, experiment_runner.run_experiment(experiment)
  File "/home

Inspect the results

In [ ]:
results_sequential[0]

In [ ]:
pd.DataFrame(results_sequential[1])

In [ ]:
results_parallel[0]

In [ ]:
pd.DataFrame(results_parallel[1])